<a href="https://colab.research.google.com/github/chethanmo/chethanmo.github.io/blob/master/keltner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas pandas-ta matplotlib yfinance
!pip install --upgrade pandas-datareader


In [138]:
import pandas as pd
import pandas_ta as ta
import yfinance as yf
from datetime import date as date
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt


In [139]:
file_path = '/content/EQUITY_L.csv'
fields = ['SYMBOL']
tickers = pd.read_csv(file_path, usecols=fields)

In [140]:
from concurrent.futures import ThreadPoolExecutor

ath_list = []
for i in range(0,len(tickers)):
  ath_list.append(tickers['SYMBOL'][i] +'.NS')

unwanted = {'INDO-RE.NS','CDSL.NS','HERIT-RE.NS','ZODIAC.NS','EMAMIPAP.NS','DIGJAMLMTD.NS','GUJRAFFIA.NS','NIRAJISPAT.NS','KALYANI.NS','ARTEMISMED.NS'}
ath_list = [e for e in ath_list if e not in unwanted]

#print (ath_list)

def get_stats(ticker):
    info = yf.Tickers(ticker).tickers[ticker].fast_info
    return ticker, info['last_price'], info['year_high'], info['two_hundred_day_average']
def paralel_func(ticker):
    with ThreadPoolExecutor(max_workers=None) as executor:
        results = list(executor.map(get_stats, ticker))
    return results

results = paralel_func(ath_list)

In [141]:
ticker_list = pd.DataFrame(results, columns=['Ticker', 'Last_price', 'ATH', 'two_EMA']) #creating the dataframe

#pd.set_option('display.max_rows', None)

ticker_list = ticker_list.query('Last_price > ATH * 0.9 and Last_price > two_EMA and Last_price < 200')
ticker_list.reset_index(drop=True, inplace=True)

In [142]:
#******* 1D *******
startdate = date.today() - relativedelta(years=4)
enddate = date.today() + relativedelta(days=1)

x = ticker_list['Ticker'].to_string(header=False,index=False).split('\n')
tickers = [','.join(ele.split()) for ele in x]

df_1d = yf.download(tickers, start= startdate, end= enddate)

#******* 1h *******
startdate = date.today() - relativedelta(days=729)
enddate = date.today() + relativedelta(days=1)

x = ticker_list['Ticker'].to_string(header=False,index=False).split('\n')
tickers = [','.join(ele.split()) for ele in x]

df_1h = yf.download(tickers, start= startdate, end= enddate, interval ='1h')

#******* 15m *******
startdate = date.today() - relativedelta(days=59)
enddate = date.today() + relativedelta(days=1)

x = ticker_list['Ticker'].to_string(header=False,index=False).split('\n')
tickers = [','.join(ele.split()) for ele in x]

df_15m = yf.download(tickers, start= startdate, end= enddate, interval ='15m')

[*********************100%***********************]  36 of 36 completed
[*********************100%***********************]  36 of 36 completed
[*********************100%***********************]  36 of 36 completed


In [143]:
pd.set_option('display.max_rows', 10)
#df

In [144]:
from time import strftime
import datetime
prev_day = (date.today() - datetime.timedelta(days=1)).strftime('%Y-%m-%d')
to_day = date.today().strftime('%Y-%m-%d')

In [145]:
def setup_func(data):
  ax_df1 = data.copy()
  df1 = pd.DataFrame()

  for ticker in tickers:
    test = ta.kc(close=ax_df1[('Close', ticker)], high=ax_df1[('High', ticker)], low=ax_df1[('Low', ticker)], length=20, scalar=1.2, mamode='ema', tr=True)
    test1 = ta.ema(close=ax_df1[('Close', ticker)], length=200).to_frame('EMA200')
    test.columns = pd.MultiIndex.from_product([test.columns, [ticker]])
    test1.columns = pd.MultiIndex.from_product([test1.columns, [ticker]])
    df1 = pd.concat([df1, test, test1], axis=1)
    df1.sort_index()
    
  ax_df1 = pd.concat([ax_df1, df1], axis=1)
  ax_df1.reset_index()
  ax_df1 = ax_df1.drop(['Adj Close','Volume'], level=0, axis=1)

  ax_df1= ax_df1.swaplevel(0, 1, 1).sort_index(axis=1)
  ax_df1 = ax_df1.tail(7)
  ax_df1 = ax_df1.T

  return ax_df1

In [146]:
ax_df_1d = setup_func(df_1d)
ax_df_1h = setup_func(df_1h)
ax_df_15m = setup_func(df_15m)

In [147]:
def keltner_stocks(data):
  count = 0
  for ticker in tickers:
    for column in data.columns:
      if (data.loc[(ticker,'EMA200'), data.columns[6]] - data.loc[(ticker,'EMA200'), data.columns[0]] > 0) and (data.loc[(ticker,'EMA200'), data.columns[6]] - data.loc[(ticker,'EMA200'), data.columns[5]] > 0):
        if data.loc[(ticker,'Close'), column] < data.loc[(ticker,'KCLe_20_1.2'), column]:
          if column.strftime('%Y-%m-%d') == to_day:
            print(ticker,column)
            #print(data.loc[ticker])
            print('\n')
            count+=1
  print('Keltner stocks: ', count)

In [148]:
keltner_stocks(ax_df_1d)

RECLTD.NS 2023-02-14 00:00:00


Keltner stocks:  1


In [149]:
keltner_stocks(ax_df_1h)


ISMTLTD.NS 2023-02-14 09:15:00+05:30


L&TFH.NS 2023-02-14 09:15:00+05:30


L&TFH.NS 2023-02-14 10:15:00+05:30


SAKSOFT.NS 2023-02-14 10:15:00+05:30


SAKSOFT.NS 2023-02-14 12:15:00+05:30


SAKSOFT.NS 2023-02-14 13:15:00+05:30


SUNFLAG.NS 2023-02-14 09:15:00+05:30


Keltner stocks:  7


In [150]:
keltner_stocks(ax_df_15m)

FINPIPE.NS 2023-02-14 15:15:00


JINDALSAW.NS 2023-02-14 14:15:00


JINDALSAW.NS 2023-02-14 14:30:00


MANAKSIA.NS 2023-02-14 15:00:00


MANAKSIA.NS 2023-02-14 15:15:00


MANOMAY.NS 2023-02-14 13:45:00


MANOMAY.NS 2023-02-14 14:00:00


MANOMAY.NS 2023-02-14 14:15:00


MANOMAY.NS 2023-02-14 15:00:00


MANOMAY.NS 2023-02-14 15:15:00


Keltner stocks:  10
